Check we have connection correct. To test this we call get a FHIR Conformance statement from the server

In [51]:
import requests

server = "http://192.168.1.67/healthconnect/cdr/fhir/r4/"
api_url = server + "metadata"
response = requests.get(api_url)
#response.json()

Find a patient. Simple search for a patient named wrexham

In [52]:
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
load_dotenv()
import os
fhir_password = os.getenv("FHIR_PASSWORD")
fhir_username = os.getenv("FHIR_USERNAME")

api_url = server + "Patient?name=wrexham"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Bundle',
 'id': '1ddbadc9-f60a-4df1-8b98-c8076429d67c',
 'type': 'searchset',
 'timestamp': '2025-12-23T06:53:08Z',
 'total': 1,
 'link': [{'relation': 'self',
   'url': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/Patient?name=wrexham'}],
 'entry': [{'fullUrl': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/Patient/18',
   'resource': {'resourceType': 'Patient',
    'address': [{'id': 'qEFQG',
      'line': ['30 PASTURE ROAD', 'BARTON-UPON-HUMBER', 'S HUMBERSIDE'],
      'period': {'start': '2025-06-24'},
      'postalCode': 'DN18 5HU',
      'use': 'home'}],
    'birthDate': '1987-02-18',
    'gender': 'unknown',
    'generalPractitioner': [{'id': 'ZGilw',
      'identifier': {'period': {'start': '2025-03-02'},
       'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
       'value': 'A20047'},
      'reference': 'Organization/1',
      'type': 'Organization'}],
    'id': '18',
    'identifier': [{'system': 'https://fhir.nhs.uk/Id/nhs-number',
      'type':

Now lets find diagnostic reports for this patient.

In [53]:
api_url = server + "DiagnosticReport?patient=18"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Bundle',
 'id': '77ec8cbf-2d7f-4988-97a3-e2d47518dea7',
 'type': 'searchset',
 'timestamp': '2025-12-23T06:53:08Z',
 'total': 1,
 'link': [{'relation': 'self',
   'url': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/DiagnosticReport?patient=18'}],
 'entry': [{'fullUrl': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/DiagnosticReport/22',
   'resource': {'resourceType': 'DiagnosticReport',
    'basedOn': [{'identifier': {'assigner': {'identifier': {'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
         'value': 'RBS'}},
       'system': 'https://nw-gmsa.github.io/Id/PLAC',
       'type': {'coding': [{'code': 'PLAC',
          'system': 'http://terminology.hl7.org/CodeSystem/v2-0203'}]},
       'value': '1000152867'},
      'type': 'ServiceRequest'}],
    'category': [{'coding': [{'code': 'GE',
        'system': 'http://terminology.hl7.org/CodeSystem/v2-0074'}]}],
    'code': {'coding': [{'code': 'R125_Cardiology',
       'display': 'PANEL: R125.1, R125.2 -

Can also return details about the organisation that created this.

In [54]:
api_url = server + "Organization/5"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Organization',
 'active': True,
 'id': '5',
 'identifier': [{'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
   'value': '699X0'}],
 'name': 'NORTH WEST GLH LED BY MANCHESTER UNIVERSITY NHS FOUNDATION TRUST',
 'partOf': {'display': 'MANCHESTER UNIVERSITY NHS FOUNDATION TRUST',
  'identifier': {'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
   'value': 'R0A'},
  'reference': 'Organization/4'},
 'type': [{'coding': [{'code': 'RO173',
     'display': 'PATHOLOGY LAB',
     'system': 'https://digital.nhs.uk/services/organisation-data-service/CodeSystem/ODSOrganisationRole'}]}],
 'meta': {'lastUpdated': '2025-12-19T11:20:46Z', 'versionId': '2'}}

Example using python to interpret the json and also using [fhirclient](https://github.com/smart-on-fhir/client-py)

In [55]:
import fhirclient.models.diagnosticreport as dr
import pandas as pd
import json

api_url = server + "DiagnosticReport?_count=50"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
responseJSON = response.json()

print(responseJSON['total'])

reports = []
reportJ = []
for entry in responseJSON['entry']:
    #print(entry['resource'])
    print(entry['resource']['resourceType'], entry['resource']['issued'] )
    report = dr.DiagnosticReport(entry['resource'])
    reportJ.append(entry['resource'])
    reports.append(report)
    for coding in report.code.coding:
        print(coding.code)

6
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
DiagnosticReport 2025-12-17T12:22:52+00:00
SANGER-Family-follow-up-Rare
R375.1
DiagnosticReport 2025-12-17T12:25:36+00:00
M80_AML_GX
M80.2
DiagnosticReport 2025-12-18T12:14:40+00:00
PCR-FLT3-ITD
M80.18
M80.21


In [56]:
print(len(reports))
df1 = pd.DataFrame([vars(s) for s in reports])
df1

6


,basedOn,category,code,conclusion,conclusionCode,effectiveDateTime,effectivePeriod,encounter,identifier,imagingStudy,...,extension,modifierExtension,text,id,implicitRules,language,meta,_server,_resolved,_owner
0,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,7,None,None,<fhirclient.models.meta.Meta object at 0x11655...,None,None,None
1,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,16,None,None,<fhirclient.models.meta.Meta object at 0x1174b...,None,None,None
2,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,22,None,None,<fhirclient.models.meta.Meta object at 0x1166e...,None,None,None
3,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,61,None,None,<fhirclient.models.meta.Meta object at 0x11721...,None,None,None
4,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,75,None,None,<fhirclient.models.meta.Meta object at 0x1173c...,None,None,None
5,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,<fhirclient.models.fhirreference.FHIRReference...,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,88,None,None,<fhirclient.models.meta.Meta object at 0x1173c...,None,None,None


In [57]:
def performer(my_list):
    performr = ""
    for item in my_list:
        performr = item.display
    return performr

def test(concept):
    code = ""
    for coding in concept.coding:
        code = coding.code
    print(code)
    return code

df1['performerN'] = df1['performer'].apply(performer)
df1['codeN'] = df1['code'].apply(test)

df1

R125_Cardiology
R125_Cardiology
R125_Cardiology
R375.1
M80.2
M80.21


,basedOn,category,code,conclusion,conclusionCode,effectiveDateTime,effectivePeriod,encounter,identifier,imagingStudy,...,text,id,implicitRules,language,meta,_server,_resolved,_owner,performerN,codeN
0,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,7,None,None,<fhirclient.models.meta.Meta object at 0x11655...,None,None,None,NHS North West Genomics,R125_Cardiology
1,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,16,None,None,<fhirclient.models.meta.Meta object at 0x1174b...,None,None,None,NHS North West Genomics,R125_Cardiology
2,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,22,None,None,<fhirclient.models.meta.Meta object at 0x1166e...,None,None,None,NHS North West Genomics,R125_Cardiology
3,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,61,None,None,<fhirclient.models.meta.Meta object at 0x11721...,None,None,None,NHS North West Genomics,R375.1
4,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,75,None,None,<fhirclient.models.meta.Meta object at 0x1173c...,None,None,None,NHS North West Genomics,M80.2
5,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,<fhirclient.models.fhirreference.FHIRReference...,[<fhirclient.models.identifier.Identifier obje...,None,...,None,88,None,None,<fhirclient.models.meta.Meta object at 0x1173c...,None,None,None,NHS North West Genomics,M80.21
